<a href="https://colab.research.google.com/github/AntonioWh1te/The-Journey/blob/main/Phase_2_1_Financial_Modeling_Prep_(FMP)_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Understanding the Task
* Goal: Fetch the closing prices of AAPL, NVDA, and TSLA over the past year, then plot them.

* Extensions
  1. Changing data frequency to daily or hourly
  2. Exploring different plot types (line, bar, candlestick).
  3. Possibly creating animations.

**Key Idea:** We'll use FMP's historical endpoints to retrieve data, parse it into Pandas, and visualize it with Matplotlib.

# Step 1: Explore the FMP API (Application Programming Interface)



In [ ]:
# For daily historical data:

url = "https://financialmodelingprep.com/api/v3"

# Depdning on the data frequency (daily or intraday), we append the correct endpoint (historical-price-full), (historical-chart/1hour)


In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Replace with your actual FMP API key
fmp_key = "YOUR_API_KEY"
ticker = "AAPL"

# Construct the URL for the quote endpoint (free version)
url = f"https://financialmodelingprep.com/api/v3/quote/{ticker}?apikey={fmp_key}"

# Make the API request
response = requests.get(url)
data = response.json()

# Convert the JSON data into a pandas DataFrame
df = pd.DataFrame(data)
print("Data retrieved from FMP:")
print(df.head())

# If data is returned, make a simple bar plot of the stock price
if not df.empty and "price" in df.columns:
    plt.figure(figsize=(8, 4))
    plt.bar(df['symbol'], df['price'], color='skyblue')
    plt.title(f"{ticker} Current Price")
    plt.xlabel("Ticker")
    plt.ylabel("Price (USD)")
    plt.show()
else:
    print("No valid data returned. Check your API key and endpoint.")


ValueError: If using all scalar values, you must pass an index